In [435]:
# !pip install git+https://github.com/psf/requests-html.git#egg=requests-html
%reset
import httpx
import re
import requests
import codecs
import time
import nltk

import threading
import pandas as pd

from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from sklearn.preprocessing import MinMaxScaler


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [436]:
HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
          }

YAHOO_CONSENT_COOKIES = None

DATA_LOCATION = 'Datasets/'

HOMEPAGE_URL = 'https://www.investopedia.com/markets-news-4427704'

VUSA_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500-companies'
SP_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500'
DATAHUB_URL = 'https://datahub.io'


YAHOO_URL = 'https://finance.yahoo.com/'

nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [437]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu') 

In [438]:
def get_yahoo_cookies():
    yahoo_cookie_driver = webdriver.Chrome(options=options)
    yahoo_cookie_driver.get(YAHOO_URL)
    time.sleep(2)
    yahoo_cookie_driver.find_element(By.CLASS_NAME, 'reject-all').click()
    time.sleep(2)
    all_cookies=yahoo_cookie_driver.get_cookies();
    cookies_dict = {}
    for cookie in all_cookies:
        cookies_dict[cookie['name']] = cookie['value']
    cookies = [{'name': cookie[0], 'value': cookie[1]} for cookie in cookies_dict.items()]
    return cookies

In [439]:
YAHOO_CONSENT_COOKIES = get_yahoo_cookies()

In [440]:
def get_response(url, cookies={}, headers=HEADERS):
    r = httpx.get(url, cookies=cookies, headers=headers, follow_redirects=True)
    soup = bs(r.content)
    return soup

In [441]:
soup = get_response(HOMEPAGE_URL, HEADERS)

In [442]:
def get_card_titles():
    titles = []
    a_tags = soup.find_all('a', class_='card')
    for i in range(len(a_tags)):
        titles.append(a_tags[i].span.text)
    return titles

def get_card_urls():
    urls = []
    a_tags = soup.find_all('a', class_='card')
    for i in range(len(a_tags)):
        urls.append(a_tags[i]['href'])
    return urls

In [443]:
titles = get_card_titles()
urls = get_card_urls()

In [444]:
investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
display(investopedia_df)

,url,title
0,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week
1,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Airline Stocks...
2,https://www.investopedia.com/fake-sec-social-m...,Fake SEC Social Media Post Puts Security Pract...
3,https://www.investopedia.com/midday-movers-oil...,"Midday Movers: Oil Jumps, Hitting Airline Stoc..."
4,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Friday: DocuSi...
...,...,...
60,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Thursday: GC E...
61,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens
62,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses: An Update to the Mod...
63,https://www.investopedia.com/crypto-related-sh...,Crypto-Related Company Stocks Jump as Digital ...


In [445]:
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text)

In [446]:
def get_url_content(urls):
    articles = []
    raw_articles = []
    for url in urls:
        soup = get_response(url)
        paragraphs = soup.find('div', class_='article-content').find_all('p')
        article = ''.join([p.get_text(separator=' ') for p in paragraphs])
        
        raw_articles.append(paragraphs)
        articles.append(clean_text(article))
    return articles, raw_articles

def get_symbols(raw_articles, href_ref):
    symbol_list = []
    for p_list in raw_articles:
        article_symbols = []
        for p in p_list:
            symbols = [a_tag.text for a_tag in p.find_all('a') if re.search(href_ref, a_tag['href'])]
            if symbols:
                for symbol in symbols:
                    article_symbols.append(symbol)
        symbol_list.append(article_symbols)
    return symbol_list

In [447]:
articles, raw_articles = get_url_content(investopedia_df['url'])

In [448]:
investopedia_df['article'] = articles

In [449]:
symbols = get_symbols(raw_articles, 'widgetsymbol')

In [450]:
investopedia_df['symbols'] = symbols

In [451]:
display(investopedia_df)

,url,title,article,symbols
0,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week,\nMarkets are closed on Monday Jan 15 for Mart...,"[MS, GS, MS, GS, SCHW, DFS, MTB, FITB, STT, WF..."
1,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Airline Stocks...,\nUS equities were mixed as earnings season go...,"[CTSH, DAL, UAL, AAL, TSLA, HTZ, WRL, ELUXY]"
2,https://www.investopedia.com/fake-sec-social-m...,Fake SEC Social Media Post Puts Security Pract...,\nThe hack this week of the Securities and Exc...,[SWI]
3,https://www.investopedia.com/midday-movers-oil...,"Midday Movers: Oil Jumps, Hitting Airline Stoc...",\nUS equities were little changed at midday as...,"[DAL, UNH, CVS, TGT, BK, T, VZ, NOC]"
4,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Friday: DocuSi...,\nHeres our daily look at three charts connect...,"[DOCU, BX, HTZ, TSLA, MS]"
...,...,...,...,...
60,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Thursday: GC E...,\nHeres our daily look at three charts connect...,"[LOPE, IOVA]"
61,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens,\nApple resumed sales of watches after an appe...,"[AAPL, SQM, TSLA, SATS, DISH]"
62,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses: An Update to the Mod...,\nIt wasnt much but US equities kept their ral...,"[KMX, MRNA, TSLA, NWSA, MSFT, NEM, LVS, DIS, A..."
63,https://www.investopedia.com/crypto-related-sh...,Crypto-Related Company Stocks Jump as Digital ...,\nStocks related to the cryptocurrency industr...,"[COIN, MARA, RIOT, MSTR]"


In [452]:
def get_datahub_csv(URL, filename):
    soup = get_response(URL)
    download_url = soup.find_all('table')[1].find_all('a')[1]['href']
    download_url = DATAHUB_URL + download_url
    with open(DATA_LOCATION + filename, 'wb') as f:
        f.write(requests.get(download_url).content)

In [453]:
get_datahub_csv(VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
get_datahub_csv(SP_DOWNLOAD_URL, 'time_data.csv')

In [454]:
vusa_df = pd.read_csv(DATA_LOCATION + 'vusa_holdings.csv')
symbols_in_vusa = vusa_df['Symbol'].tolist()

In [455]:
investopedia_df['appears_in_vusa'] = investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))
investopedia_df = investopedia_df[investopedia_df['appears_in_vusa']]

,url,title,article,symbols,appears_in_vusa
0,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week,\nMarkets are closed on Monday Jan 15 for Mart...,"[MS, GS, MS, GS, SCHW, DFS, MTB, FITB, STT, WF...",True
1,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Airline Stocks...,\nUS equities were mixed as earnings season go...,"[CTSH, DAL, UAL, AAL, TSLA, HTZ, WRL, ELUXY]",True
3,https://www.investopedia.com/midday-movers-oil...,"Midday Movers: Oil Jumps, Hitting Airline Stoc...",\nUS equities were little changed at midday as...,"[DAL, UNH, CVS, TGT, BK, T, VZ, NOC]",True
4,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Friday: DocuSi...,\nHeres our daily look at three charts connect...,"[DOCU, BX, HTZ, TSLA, MS]",True
5,https://www.investopedia.com/dow-jones-today-0...,"Markets News, Jan. 12, 2024: Stocks Muted Afte...",\nStocks closed Friday little changed after bi...,"[VZ, CVX, MSFT, AAPL, UNH, BA, JPM, BK, NOC, L...",True
6,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens,\nShares of JPMorgan Chase rose after the bank...,"[JPM, BAC, WFC, C, DAL, XOM, BP, CVX, MARA, RI...",True
7,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Netflix Lifted...,\nThe SP 500 declined by 007 after inflation d...,"[NFLX, PARA, WBD, CRM, VLO, EL, T]",True
8,https://www.investopedia.com/midday-movers-net...,Midday Movers: Netflix Gains as Ad-Backed Plan...,\nUS equities were down at midday as the late...,"[NFLX, WBD, PARA, CHK, SWN, VLO, TSLA, C]",True
9,https://www.investopedia.com/dow-jones-today-0...,"Markets News, Jan. 11, 2024: Stocks Close Litt...",\n UpdateJan 12 2024 For todays live markets ...,"[JPM, BAC, C, CRM, CVX, MSFT, AAPL, VZ, T, BA,...",True
10,https://www.investopedia.com/3-charts-for-inve...,3 Charts For Investors to Watch Thursday: Coin...,\nHeres our daily look at three charts linked ...,"[COIN, PARA, TCEHY, IRBT, AMZN]",True


In [457]:
def get_article_links(driver):
    links = driver.find_elements(By.TAG_NAME, 'a')
    return links

In [458]:
def get_yahoo_news_articles_df():
    yahoo_news_driver = webdriver.Chrome(options=options)

    yahoo_news_driver.get(YAHOO_URL)
    time.sleep(1)
    yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
    actions = ActionChains(yahoo_news_driver)
    for i in range(1500):
        actions.scroll_by_amount(0, 500)
    actions.perform()
    actions.reset_actions()

    links = get_article_links(yahoo_news_driver)
    df = pd.DataFrame({'title': [link.text for link in links], 'url': [link.get_attribute('href') for link in links], 'classes': [link.get_attribute('class') for link in links]})
    yahoo_news_driver.quit()
    df = df[df['url'] \
            .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
            .drop_duplicates(subset='url', keep='last')\
            .reset_index(drop=True)
    return df

In [459]:
yahoo_news_df = get_yahoo_news_articles_df()

In [460]:
display(yahoo_news_df)

,title,url,classes
0,Redesigned Apple Watches not subject to import...,https://finance.yahoo.com/news/redesigned-appl...,js-content-viewer Fw(b) Td(n) wafer-destroyed
1,Boeing to add further inspections for 737 MAX\...,https://finance.yahoo.com/news/boeing-add-furt...,js-content-viewer Fw(b) Td(n) wafer-destroyed
2,Microsoft co-founder Bill Gates: I am concerne...,https://finance.yahoo.com/news/microsoft-co-fo...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...
3,One key reason inflation is cooling: Immigrant...,https://finance.yahoo.com/news/one-key-reason-...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...
4,Apple could have a new problem: Uncle Sam,https://finance.yahoo.com/news/apple-could-hav...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...
...,...,...,...
216,Buying the New Bitcoin ETFs? 3 Things to Keep ...,https://finance.yahoo.com/news/buying-bitcoin-...,js-content-viewer wafer-caas Td(n) D(ib) Va(t)...
217,Investors bet via options that India's rupee w...,https://finance.yahoo.com/news/investors-bet-v...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
218,1 Growth Stock Down 81% to Buy Right Now,https://finance.yahoo.com/news/1-growth-stock-...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
219,1 Top Growth Stock Down 80% That's a Screaming...,https://finance.yahoo.com/news/1-top-growth-st...,js-content-viewer wafer-caas Td(n) D(ib) Va(t)...


In [461]:
def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

In [462]:
def scrape_articles(urls):
    articles = []
    raw_articles = []
    yahoo_driver = webdriver.Chrome(options=options)
    yahoo_driver.get(YAHOO_URL)
    yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

    for index, url in enumerate(urls):
        yahoo_driver.get(url)
        try:
            yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
        except:
            pass
        article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
        article_by_paragraph = [p.text for p in article_p_elements if p.text!='']
        
        raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
        article = " ".join(article_by_paragraph)
        clean_article = clean_text(article)
        articles.append(clean_article)
        raw_articles.append(raw_article)
        progress_bar(index+1, len(urls))
    
        
    yahoo_driver.quit()
    
    return articles, raw_articles

In [ ]:
yahoo_news_articles, raw_yahoo_news_articles = scrape_articles(yahoo_news_df['url'])

In [ ]:
yahoo_news_df['article'] = yahoo_news_articles

In [ ]:
yahoo_news_df

In [ ]:
def combine_dfs(investopedia_df, yahoo_df):
    df1 = investopedia_df.drop(['symbols', 'appears_in_vusa'], axis=1)
    df2 = yahoo_df.drop('classes', axis=1)
    article_df = pd.concat([df1, df2])
    article_df = article_df.reset_index(drop=True)
    return article_df

In [ ]:
df = combine_dfs(investopedia_df, yahoo_news_df)

In [ ]:
def clean_and_tokenize(articles):
    lemmatizer = WordNetLemmatizer()
    
    tokenized_articles = []
    for article in articles:
        tokenized_article = nltk.word_tokenize(article)
        tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
        tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]
        
        tokenized_articles.append(tokenized_article)
        
    return tokenized_articles

In [ ]:
df['tokenized_article'] = clean_and_tokenize(df['article'])

In [ ]:
df

In [ ]:
analyzer = SentimentIntensityAnalyzer()
scaler = MinMaxScaler()
df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in df['article']]
df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in df['article']]

In [ ]:
df